<a href="https://colab.research.google.com/github/uteyechea/neural-network-from-scratch/blob/main/nn_model_2layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 1

##1.1 Import all necessary dependencies

In [ ]:
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix

##1.2 Prepare the dataset

In [ ]:
#Binary classifier toy data

# Three training examples with two features x1 and x2.
X=np.array([
    [1,-1,0],
    [2,2,-1]
])

print('# Training examples, m: {}'.format(X.shape[1]))
print('# of features, n_x: {}'.format(X.shape[0]))
#Labels for each of the three training examples.
Y=np.array([
            [1,0,0],
])

# Training examples, m: 3
# of features, n_x: 2


In [ ]:
# Visualize the data
fig=go.Figure(data=go.Scatter(x=X[0,:],
                                y=X[1,:],
                                mode='markers',
                                marker_color=Y[0,:],
                                text=Y[0,:]))
fig.update_layout(title='Binary classification training samples')
fig.show()

#Part 2

#2.1 Define layer sizes

In [ ]:
def layer_sizes(X, Y):
    """
    Arguments:
    X -- input dataset of shape (number of features, number of training examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer, i. e., number of features
    n_h -- the size of the hidden layer, i. e., number of neurons
    n_y -- the size of the output layer, i. e., label
    """
    
    n_x = X.shape[0] # size of input layer, i. e. number of features
    n_h = 3 # user defined number of neurons inside the one and only layer
    n_y = Y.shape[0] # size of output layer, i. e. number of possible labels
    
    return (n_x, n_h, n_y)

#2.2 Initialize model parameters

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer, i. e., number of features
    n_h -- size of the hidden layer, i. e., number of neurons
    n_y -- size of the output layer, i. e., label
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h) #if Second layer is the output ot the NN.
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    W1 = np.random.randn(n_h,n_x) * 0.01 # 0.01 it is usually recommended to start with small weights, as long as they are different than zero
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

#2.3 Implement forward propagation

In [ ]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m) #where m is the number of training examples
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    Z1 = W1 @ X + b1 #(n_h,n_x) @ (n_x,m) + (n_h,1) = (n_h,m)
    A1 = np.tanh(Z1) #(n_h,m) selected activation function for the hidden layer neurons
    Z2 = W2 @ A1 + b2 
    A2 = 1 / (1 + np.exp(-Z2)) #NN output, later we will round this value to 0/1, thus the sigmoid function fits the expected output map just fine
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

### 2.2.1 Compute Cost

In [ ]:
def compute_cost(A2, Y):
    """
    Computes the cross-entropy cost. It is prefered to use this cost function given it is stricly convex 
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (output size, number of examples)
    Y -- "true" labels vector of shape (output size, number of examples)
    
    Returns:
    cost -- cross-entropy cost
    
    """
    
    m = Y.shape[1] # get the number of training examples

    # Compute the cross-entropy cost
    logprobs = Y*np.log(A2) + (1-Y) * np.log(1-A2) # (n_y,m) * (n_h,m) + ... = (n_h,m)
    cost = -(1/m)*np.sum(logprobs,axis=1,keepdims=True) # sum the cost over all training examples
    
    cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect
                                    
    assert(isinstance(cost, float))

    return cost

#Part 3

#3.1 Implement Backpropagation

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1] #get number of training examples
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    W1 = parameters['W1']
    W2 = parameters['W2']
        
    # Retrieve also A1 and A2 from dictionary "cache".
    A1 = cache['A1']
    A2 = cache['A2']
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    dZ2 = A2-Y
    dW2 = (1/m)*dZ2@A1.T #(n_h,m) @ (n_h,m) = (n_h,n_h)
    db2 = (1/m)*np.sum(dZ2,axis=1,keepdims=True)
    dZ1 = W2.T@dZ2 * (1 - np.power(A1, 2)) 
    # (n_h,n_x).T @ (n_h,m) * (n_h,m) - (n_y,m) = (n_x,m) * (n_h,m) Broadcasting
    dW1 = (1/m)*dZ1@X.T #(n_h,m) * (n_h,m) = (n_h,m)
    db1 = (1/m)*np.sum(dZ1,axis=1,keepdims=True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

#Part 4

#4.1 Update parameters

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    """
    Updates parameters using the gradient descent update rule. Ex: w= w-alpha*dw, b= b-alpha*db
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Retrieve each gradient from the dictionary "grads"
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    # Update rule for each parameter
    W1 = W1-learning_rate*dW1
    b1 = b1-learning_rate*db1
    W2 = W2-learning_rate*dW2
    b2 = b2-learning_rate*db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

#Part 5

##5.1 Build NN model - 2 layers

In [ ]:
def nn_model(X, Y, n_h, num_iterations = 10000, learning_rate=0.01, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    learning_rate -- step size while implementing gradient descent
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A, cache".
        A, cache = forward_propagation(X, parameters)
        
        # Cost function. Inputs: "A, Y, parameters". Outputs: "cost".
        cost = compute_cost(A, Y)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

#5.2 Make function to generate predictions

In [ ]:
def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m), could be test set, validation set or new observations
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    A, cache = forward_propagation(X, parameters)
    predictions = np.around(A,0) # integer class labels 0/1
    
    
    return predictions

# Part 6: Run model

In [ ]:
# Build a model with a n_h-dimensional hidden layer
parameters = nn_model(X, Y, n_h = 3, num_iterations = 10000, learning_rate=0.01, print_cost=True)

Cost after iteration 0: 0.693155
Cost after iteration 1000: 0.510925
Cost after iteration 2000: 0.080845
Cost after iteration 3000: 0.030955
Cost after iteration 4000: 0.018098
Cost after iteration 5000: 0.012565
Cost after iteration 6000: 0.009549
Cost after iteration 7000: 0.007668
Cost after iteration 8000: 0.006390
Cost after iteration 9000: 0.005468


In [ ]:
# Print accuracy
predictions = predict(parameters, X)
print ('Accuracy: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')

Accuracy: 100%


In [ ]:
predictions_table=np.array([
                            predictions,
                            Y
])

In [ ]:
# Confusion matrix
#sklearn.metrics.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)
confusion_matrix(Y.T,predictions.T)

array([[2, 0],
       [0, 1]])